<a href="https://colab.research.google.com/github/SushmithaUW/Homesickness-Prediction/blob/master/Association.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import random
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics
from sklearn import preprocessing


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


In [0]:
#load the data
train = pd.read_csv('data.csv')
train.head()

#remove the unwanted columns and rows containing null values
train = train.drop('Timestamp',axis=1)
train = train.drop('Unnamed: 13', axis=1)
train = train.drop('Unnamed: 14', axis=1)


#Make the column header readable
train.columns=['Age', 'Gender', 'International_Student', 'Educational_level', 'Course_difficulty', 'Networking', 'Financial_aid', 'Friends', 'Siblings', 'Howmetown_visits', 'Cooking', 'Homesickness']
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 12 columns):
Age                      116 non-null object
Gender                   116 non-null object
International_Student    115 non-null object
Educational_level        116 non-null object
Course_difficulty        116 non-null int64
Networking               116 non-null object
Financial_aid            116 non-null object
Friends                  116 non-null object
Siblings                 116 non-null object
Howmetown_visits         116 non-null object
Cooking                  116 non-null object
Homesickness             116 non-null object
dtypes: int64(1), object(11)
memory usage: 11.0+ KB


In [0]:
#There are same values (I don't know, Thirce, Twice, etc,) amoung columns
#We will try to uniquely represent the values of each of the column. 
df = []
df = train['Course_difficulty']
df.replace(1,'Easy',True)
df.replace(2,'Easy',True)
df.replace(3,'Moderate',True)
df.replace(4,'Moderate',True)
df.replace(5,'Difficult',True)
train['Course_difficulty'] = df
train['Course_difficulty'].unique()

df = []
df = train['Networking']
df.replace("I don't know",'Does not know about netwrking',True)
df.replace('Once','Networking happens once',True)
df.replace('Twice','Networking happens twice',True)
df.replace('Thrice or more','Networking happens often',True)
train['Networking'] = df
train['Networking'].unique()

df = []
df = train['Financial_aid']
df.replace("I don't know",'Does not know about financial aid',True)
df.replace('Yes','University provides financial aid',True)
df.replace('No','University does not provide financial aid',True)
df.replace('Thrice or more','Networking happens often',True)
train['Financial_aid'] = df
train['Financial_aid'].unique()



df = []
df = train['Friends']
df.replace('4 or more','Extrovert',True)
df.replace('3 or more','Friendly',True)
df.replace('0','Lonely',True)
df.replace('2','Introvert',True)
df.replace('5 or more','Extrovert',True)
df.replace('1','Lonely',True)
df.replace('3','Friendly',True)
df.replace('4','Extrovert',True)
train['Friends'] = df
train['Friends'].unique()

df = []
df = train['Siblings']
df.replace('0','Single Child',True)
df.replace('1','One sibling',True)
df.replace('2','Two siblings',True)
df.replace('3','Three siblings',True)
df.replace('4','Large family',True)
df.replace('5 or more','Large family',True)
train['Siblings'] = df
train['Siblings'].unique()

df = []
df = train['Howmetown_visits']
df.replace('Once','Visits once',True)
df.replace('Never','Never visits',True)
df.replace('Twice','Visits twice',True)
df.replace('Thrice or more','Visits often',True)
train['Howmetown_visits'] = df
train['Howmetown_visits'].unique()


df = []
df = train['Cooking']
df.replace('Thrice','Cooks thrice',True)
df.replace('Thrice or more','Cooks very often',True)
df.replace('2 to 3 times','Cooks twice',True)
df.replace('4 to 6 times','Cooks very often',True)
df.replace('Never','Never cooks',True)
df.replace('Everyday','Cooks everyday',True)
train['Cooking'] = df
train['Cooking'].unique()

for column in train.columns:
    print (column)
    print (train[column].unique())
    print ("\n")


Age
['18-25' 'Below 18' 'Above 30' '26-30']


Gender
['Female' 'Male' 'Other']


International_Student
[nan 'Yes' 'No']


Educational_level
['Graduate' 'Undergraduate' 'PhD']


Course_difficulty
['Moderate' 'Easy' 'Difficult']


Networking
['Does not know about netwrking' 'Networking happens once'
 'Networking happens twice' 'Networking happens often']


Financial_aid
['Does not know about financial aid' 'University provides financial aid'
 'University does not provide financial aid']


Friends
['Extrovert' 'Friendly' 'Lonely' 'Introvert']


Siblings
['One sibling' 'Two siblings' 'Single Child' 'Three siblings'
 'Large family']


Howmetown_visits
['Visits once' 'Never visits' 'Visits twice' 'Visits often']


Cooking
['Cooks thrice' 'Cooks very often' 'Cooks twice' 'Cooks everyday'
 'Never cooks']


Homesickness
['Twice' '4 to 6 times' '2 to 3 times' 'Everyday' 'Never']




In [0]:
#Filter out the local students and remove the nan records
train = train.dropna()
local_students = train[ train['International_Student'] == 'No' ].index
train.drop(local_students , inplace=True)
train['International_Student'].unique()
train = train.drop('International_Student', axis=1)



In [0]:
data = train.values.tolist()
oht = TransactionEncoder()
oht_ary = oht.fit(data).transform(data)
df = pd.DataFrame(oht_ary, columns=oht.columns_)
print(df)


    18-25  2 to 3 times  26-30  ...  Visits often  Visits once  Visits twice
0    True         False  False  ...         False        False         False
1   False         False  False  ...         False        False          True
2    True          True  False  ...         False        False         False
3    True         False  False  ...         False        False         False
4   False         False  False  ...         False        False         False
5    True          True  False  ...         False         True         False
6    True         False  False  ...         False        False         False
7    True         False  False  ...         False         True         False
8   False          True   True  ...         False        False         False
9    True         False  False  ...         False        False         False
10  False         False   True  ...         False        False         False
11  False         False   True  ...         False        False          True

In [0]:
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)
print(frequent_itemsets)

     support                                       itemsets
0   0.581818                                        (18-25)
1   0.672727                (Does not know about netwrking)
2   0.581818                                     (Everyday)
3   0.727273                                    (Extrovert)
4   0.781818                                     (Graduate)
5   0.672727                                         (Male)
6   0.800000                                     (Moderate)
7   0.581818                                 (Never visits)
8   0.654545                                  (One sibling)
9   0.581818            (University provides financial aid)
10  0.509091                              (18-25, Moderate)
11  0.509091     (Extrovert, Does not know about netwrking)
12  0.563636      (Graduate, Does not know about netwrking)
13  0.545455      (Moderate, Does not know about netwrking)
14  0.581818                          (Extrovert, Graduate)
15  0.636364                          (E

In [0]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.9)
print(rules[['antecedents', 'consequents', 'support', 'confidence']])

                antecedents consequents   support  confidence
0                (Graduate)  (Moderate)  0.709091    0.906977
1            (Never visits)  (Graduate)  0.545455    0.937500
2     (Extrovert, Graduate)  (Moderate)  0.545455    0.937500
3  (Graduate, Never visits)  (Moderate)  0.509091    0.933333
4  (Never visits, Moderate)  (Graduate)  0.509091    1.000000
